In [18]:
import pandas as pd
df = pd.read_csv('../data/News Articles/processed/trump_analysed.csv', encoding='ISO-8859-1')
df1 = pd.read_excel('../data/Stock Data/SPY 1min.xlsx')

print(df.head())
print(df1.head())

         datetime                                               text  \
0  7/9/2025 20:29  I am pleased to announce that I am directing o...   
1  7/9/2025 20:29  I am announcing a 50% TARIFF on Copper, effect...   
2  7/9/2025 20:13  HAPPY BIRTHDAY TO SENATOR LINDSEY GRAHAM! He i...   
3  7/9/2025 20:12  Senator Tom Cotton is working incredibly hard ...   
4  7/9/2025 20:11  Senator Roger Marshall is an incredibly strong...   

   isRetweet                                     processed_text  \
0      False  great transportation sean duffy interim nasa s...   
1      False  tariff copper national security copper aircraf...   
2      False  lindsey great south carolina lindsey big make ...   
3      False  tom cotton great arkansas won u.s.a. american ...   
4      False  roger marshall kansas won medical doctor u.s. ...   

                                         topics  \
0                     ['Transport & Logistics']   
1  ['Corporate Sector', 'Geopolitics & Policy']   
2          

In [19]:
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df = df.sort_values(by='datetime')

df1['Time at end of bar'] = pd.to_datetime(df1['Time at end of bar'])
df1.set_index('Time at end of bar', inplace=True)
df1 = df1.sort_values(by='Time at end of bar')

In [20]:
df['sentiment'] = df['sentiment'].apply(lambda x:  1 if x == 'POSITIVE' else -1)

df['sentiment']

datetime
2025-01-03 13:25:00    1
2025-01-03 13:31:00   -1
2025-01-03 14:59:00    1
2025-01-03 16:16:00    1
2025-01-03 18:27:00    1
                      ..
2025-07-09 20:11:00    1
2025-07-09 20:12:00    1
2025-07-09 20:13:00    1
2025-07-09 20:29:00   -1
2025-07-09 20:29:00    1
Name: sentiment, Length: 1544, dtype: int64

In [21]:
start_date = df.index[0]
end_date = df1.index[-1]

In [22]:
# BT strat funciton

import backtrader as bt

class DebugStrategy(bt.Strategy):
    params = ()

    def __init__(self):
        pass

    def next(self):
        pass

def run(Strategy = DebugStrategy, initial_cash = 10000, commission = 0.001):
    cerebro = bt.Cerebro()

    data = bt.feeds.PandasData(dataname=df1, plot=True, fromdate=start_date, todate = end_date)
    cerebro.adddata(data, name='price')

    class sentiment(bt.feeds.PandasData):
        lines = ('label', 'score')
        params = (
            ('datetime', None),  # Use DataFrame index as datetime
            ('label', 'sentiment'),
            ('score', 'sentiment_score')
        )

    data1 = sentiment(dataname=df, plot=False, fromdate = start_date, todate = end_date)
    cerebro.adddata(data1, name='sentiment')

    cerebro.addstrategy(Strategy)

    cerebro.broker.setcash(initial_cash)
    cerebro.broker.setcommission(commission=commission)

    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

    results = cerebro.run()
    strategy = results[0]

    print(f'Initial Portfolio Value: {initial_cash}, commission: {commission}')
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Sharpe Ratio:', strategy.analyzers.sharpe.get_analysis()['sharperatio'])
    print('Total Return:', strategy.analyzers.returns.get_analysis()['rtot'])
    print('Total Trades:', strategy.analyzers.trades.get_analysis()['total']['total'])

    # Plot result
    # fig = cerebro.plot(style="candlestick")[0][0]
    # fig.savefig('data/.png')

In [23]:
run()

Initial Portfolio Value: 10000, commission: 0.001
Final Portfolio Value: 10000.00
Sharpe Ratio: None
Total Return: 0.0
Total Trades: 0


In [ ]:
class TimedExitStrategy(bt.Strategy):
    params = (
        ('hold_bars', 15),  # Hold position for N bars
        ('size', 1),        # Number of shares/contracts
    )

    def __init__(self):
        self.open_trades = []
        self.trade_id = 0

    def next(self):
        current_bar = len(self)

        # === 1. Check for trade exit (if enough bars passed)
        for trade in self.open_trades[:]:  # Iterate over a copy
            if current_bar - trade['entry_bar'] >= self.p.hold_bars:
                if trade['direction'] == 'long':
                    self.sell(size=self.p.size)
                elif trade['direction'] == 'short':
                    self.buy(size=self.p.size)
                print(f"Closed {trade['direction'].upper()} trade #{trade['id']} at bar {current_bar}")
                self.open_trades.remove(trade)

        # === 2. Entry logic (multiple trades allowed)
        if self.datas[1].score[0] > 0.9964:
            if self.datas[1].label[0] == 1:
                self.buy(size=self.p.size)
                self.open_trades.append({
                    'entry_bar': current_bar,
                    'direction': 'long',
                    'id': self.trade_id
                })
                print(f"Opened LONG trade #{self.trade_id} at bar {current_bar}")
            else:
                self.sell(size=self.p.size)
                self.open_trades.append({
                    'entry_bar': current_bar,
                    'direction': 'short',
                    'id': self.trade_id
                })
                print(f"Opened SHORT trade #{self.trade_id} at bar {current_bar}")
            self.trade_id += 1

run(TimedExitStrategy)

In [ ]:
class ParallelTimedExit(bt.Strategy):
    params = (
        ('hold_bars', 5),  # Hold each position for X bars
        ('size', 1),        # Size of each trade
    )

    def __init__(self):
        self.trades = []     # Track all open trades
        self.trade_id = 0    # Unique ID for each trade

    def next(self):
        bar_idx = len(self)

        # === Check and close expired trades ===
        for trade in self.trades[:]:
            if bar_idx - trade['entry_bar'] >= self.p.hold_bars:
                if trade['direction'] == 'long':
                    self.sell(size=trade['size'])
                else:
                    self.buy(size=trade['size'])

                print(f"[{bar_idx}] CLOSED {trade['direction'].upper()} trade #{trade['id']} | "
                      f"Entry: {trade['entry_price']:.2f} → Exit: {self.data.close[0]:.2f}")

                self.trades.remove(trade)

        # === Entry Logic (replace with your signal) ===
        if self.datas[1].score[0] >= 0.9964:  # Dummy long signal
            if self.datas[1].label[0] == 1:
                self.buy(size=self.p.size)
                self.trades.append({
                    'id': self.trade_id,
                    'direction': 'long',
                    'entry_bar': bar_idx,
                    'entry_price': self.data.close[0],
                    'size': self.p.size
                })
                print(f"[{bar_idx}] ENTERED LONG trade #{self.trade_id} @ {self.data.close[0]:.2f}")
                self.trade_id += 1
            
            else:
                self.sell(size=self.p.size)
                self.trades.append({
                    'id': self.trade_id,
                    'direction': 'short',
                    'entry_bar': bar_idx,
                    'entry_price': self.data.close[0],
                    'size': self.p.size
                })
                print(f"[{bar_idx}] ENTERED SHORT trade #{self.trade_id} @ {self.data.close[0]:.2f}")
                self.trade_id += 1
run(ParallelTimedExit)

In [30]:
class TimedSentimentStrategy(bt.Strategy):
    params = (
        ('hold_bars', 5),  # Hold for 5 bars (≈5 mins if data is minute-level)
        ('size', 1),
    )

    def __init__(self):
        self.open_trades = []
        self.trade_id = 0

    def next(self):
        current_bar = len(self)
        price_dt = self.datas[0].datetime.datetime(0)
        sentiment_dt = self.datas[1].datetime.datetime(0)

        # === 1. Close any expired trades
        for trade in self.open_trades[:]:
            if current_bar - trade['entry_bar'] >= self.p.hold_bars:
                if trade['direction'] == 'long':
                    self.sell(size=self.p.size)
                elif trade['direction'] == 'short':
                    self.buy(size=self.p.size)
                print(f"[{price_dt}] Closed {trade['direction'].upper()} trade #{trade['id']}")
                self.open_trades.remove(trade)

        # === 2. Check sentiment signal (only if datetimes match)
        if price_dt == sentiment_dt:
            score = self.datas[1].score[0]
            label = self.datas[1].label[0]

            # Example logic: buy if label == 1 and score is high
            if label == 1 and score >= 0.9966:
                self.buy(size=self.p.size)
                self.open_trades.append({
                    'entry_bar': current_bar,
                    'direction': 'long',
                    'id': self.trade_id
                })
                print(f"[{price_dt}] Opened LONG trade #{self.trade_id}")
                self.trade_id += 1

            # Example short logic
            elif label == -1 and score >= 0.9966:
                self.sell(size=self.p.size)
                self.open_trades.append({
                    'entry_bar': current_bar,
                    'direction': 'short',
                    'id': self.trade_id
                })
                print(f"[{price_dt}] Opened SHORT trade #{self.trade_id}")
                self.trade_id += 1

run(TimedSentimentStrategy)

[2025-01-03 13:31:00] Opened SHORT trade #0
[2025-01-03 13:36:00] Closed SHORT trade #0
[2025-01-05 18:10:00] Opened SHORT trade #1
[2025-01-05 18:15:00] Closed SHORT trade #1
[2025-01-06 08:29:00] Opened LONG trade #2
[2025-01-06 08:34:00] Closed LONG trade #2
[2025-01-06 08:34:00] Opened LONG trade #3
[2025-01-06 08:39:00] Closed LONG trade #3
[2025-01-06 09:15:00] Opened SHORT trade #4
[2025-01-06 09:20:00] Closed SHORT trade #4
[2025-01-06 18:09:00] Opened LONG trade #5
[2025-01-06 18:14:00] Closed LONG trade #5
[2025-01-06 20:10:00] Opened LONG trade #6
[2025-01-06 20:15:00] Closed LONG trade #6
[2025-01-07 13:57:00] Opened LONG trade #7
[2025-01-07 14:02:00] Closed LONG trade #7
[2025-01-08 14:25:00] Opened SHORT trade #8
[2025-01-08 14:30:00] Closed SHORT trade #8
[2025-01-08 15:03:00] Opened SHORT trade #9
[2025-01-08 15:08:00] Closed SHORT trade #9
[2025-01-09 18:38:00] Opened LONG trade #10
[2025-01-09 18:42:00] Closed LONG trade #10
[2025-01-09 19:09:00] Opened LONG trade #1

In [ ]:
from IPython.display import Image
Image('data/.png')